## Paso 1: Importar las librerías necesarias

In [80]:
import tensorflow as tf

from tensorflow.keras.datasets import imdb

In [81]:
tf.__version__

'2.12.0'

## Paso 2: Pre procesado de datos


### Configurar parámetros del dataset

In [82]:
number_of_words = 20000 #n palabras mas frecuentes
max_len = 100 #numero de palabras, en caso de que sea menor se acompleta

### Carga del dataset de IMDB

In [83]:
#Obtenemos las 25k reviews, dentro de cada review tenemos una lista de numeros, al poner num_words = n, entonces en dicha lista los numeros mayores a n seran remplazados por otro numero (siempre el mismo)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

In [84]:
len(X_train[0])

218

### Cortar secuencias de texto de la misma longitud

In [85]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [86]:
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

In [87]:
len(X_train[0])

100

### Configurar parámetros de la capa de Embedding

In [88]:
vocab_size = number_of_words
vocab_size

20000

In [89]:
embed_size = 256

## Paso 3: Construir la Red Neuronal Recurrente

### Definir el modelo

In [108]:
model = tf.keras.Sequential()

### Añadir la capa de embedding

In [109]:
model.add(tf.keras.layers.Embedding(vocab_size, embed_size, input_shape=(X_train.shape[1],)))#Toma los textos. Vector de palabras. Param: n palabras, dimension de salida

### Añadir la capa de LSTM

- unidades: 256
- función de activación: tanh

In [110]:
model.add(tf.keras.layers.LSTM(units=256, activation='tanh',return_sequences=True))

In [111]:
model.add(tf.keras.layers.Dropout(rate=0.2))

In [112]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

### Añadir la capa totalmente conectada de salida

- unidades: 1
- función de activación: sigmoid

In [113]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compilar el modelo

In [114]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])#rmsprop para recurrentes

In [115]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 256)          5120000   
                                                                 
 lstm_13 (LSTM)              (None, 100, 256)          525312    
                                                                 
 dropout_1 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_14 (LSTM)              (None, 128)               197120    
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 5,842,561
Trainable params: 5,842,561
Non-trainable params: 0
_________________________________________________________________


### Entrenar el modelo

In [116]:
model.fit(X_train, y_train, epochs=6, batch_size=256)

Epoch 1/6
98/98 [==============================] - 202s 2s/step - loss: 0.6519 - accuracy: 0.6001
Epoch 2/6
98/98 [==============================] - 281s 3s/step - loss: 0.4345 - accuracy: 0.8018
Epoch 3/6
98/98 [==============================] - 249s 3s/step - loss: 0.3507 - accuracy: 0.8516
Epoch 4/6
98/98 [==============================] - 301s 3s/step - loss: 0.2999 - accuracy: 0.8772
Epoch 5/6
98/98 [==============================] - 264s 3s/step - loss: 0.2679 - accuracy: 0.8939
Epoch 6/6
98/98 [==============================] - 272s 3s/step - loss: 0.2364 - accuracy: 0.9076


### Evaluar el modelo

In [ ]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

468/782 [================>.............] - ETA: 1:13 - loss: 0.3786 - accuracy: 0.8447

In [ ]:
print("Test accuracy: {}".format(test_acurracy))

## Paso 4: Guardar el modelo

In [94]:
model.save("modelo_rnr.h5")